## Using Python + Github API

### First day: exploring the PyGithub module

In this lesson I will show you how to use [PyGithub](https://github.com/PyGithub/PyGithub) to retrieve public data from Github profiles. Secondly I will show you how to automatically create a gist. 

Note that there are other Github libraries, [for example GitPython](https://www.fullstackpython.com/blog/first-steps-gitpython.html), I used PyGithub for multiple occasions though, and although the docs can be improved, it works pretty well. The good news is that this gap allows me to show you some of Python's help and debug features :)

#### Getting ready

If you watched my appendix setup video you should have all the requirements already installed in your virtual env. If not, [create one first](https://pybit.es/the-beauty-of-virtualenv.html), activate it, then `pip install pygithub`.

#### Example 1: retrieve a user's popular repos

In [1]:
from collections import namedtuple
import os

from github import Github, InputFileContent

Let's start with public access, for that you don't have to pass any user/password or auth token into the `Github` constructor:

In [2]:
gh = Github()
gh

Note that using the public API (no auth) you are pretty limited with the amount of calls per hour (first int is calls left, second int is the max amount per hour):

In [3]:
gh.rate_limiting

(60, 60)

As we will see later using the GH API with a login or auth token this limit increases to 5000 calls, more info [here](https://developer.github.com/v3/#rate-limiting).

Let's make a _pybites_ user object:

In [11]:
pb = gh.get_user('pybites')
pb

NamedUser(login="pybites")

#### Pro tip: getting help in Python

Let's use the Github API to get the most popular repos for a user. Popularity is defined as the numebr of star(gazers) a repo obtained during its lifetime.

When working with _pygithub_ expect to do some object inspection on the fly, here is where you might want to learn about [Python's help methods](https://pybit.es/python-help.html):

In [4]:
!pydoc github

'pydoc' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
help(pb)

Help on NamedUser in module github.NamedUser object:

class NamedUser(github.GithubObject.CompletableGithubObject)
 |  NamedUser(requester, headers, attributes, completed)
 |  
 |  This class represents NamedUsers. The reference can be found here https://developer.github.com/v3/users/#get-a-single-user
 |  
 |  Method resolution order:
 |      NamedUser
 |      github.GithubObject.CompletableGithubObject
 |      github.GithubObject.GithubObject
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __hash__(self)
 |      Return hash(self).
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  get_events(self)
 |      :calls: `GET /users/:user/events <http://developer.github.com/v3/activity/events>`_
 |      :rtype: :class:`github.PaginatedList.PaginatedList` of :class:`github.Event.Event`
 |  
 |  get_followers(self)
 |      :calls: `GET /users/:user/followers <http://developer.github.com/v3/users/followers>`_
 |  

In [7]:
dir(pb)

['CHECK_AFTER_INIT_FLAG',
 '_CompletableGithubObject__complete',
 '_CompletableGithubObject__completed',
 '_GithubObject__makeSimpleAttribute',
 '_GithubObject__makeSimpleListAttribute',
 '_GithubObject__makeTransformedAttribute',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_avatar_url',
 '_bio',
 '_blog',
 '_collaborators',
 '_company',
 '_completeIfNeeded',
 '_completeIfNotSet',
 '_contributions',
 '_created_at',
 '_disk_usage',
 '_email',
 '_events_url',
 '_followers',
 '_followers_url',
 '_following',
 '_following_url',
 '_gists_url',
 '_gravatar_id',
 '_headers',
 '_hireable',
 '_html_url',
 '_id',
 '_identity',
 '_initAttributes',
 '_location',
 '_login',
 '

In [8]:
help(pb.get_repos)

Help on method get_repos in module github.NamedUser:

get_repos(type=NotSet) method of github.NamedUser.NamedUser instance
    :calls: `GET /users/:user/repos <http://developer.github.com/v3/repos>`_
    :param type: string
    :rtype: :class:`github.PaginatedList.PaginatedList` of :class:`github.Repository.Repository`



Now it becomes a lot easier! Let's use a namedtuple to make our code more readable. I could use a generator but I want to sort on number of stars so I collect all repos in a list, then use `sorted` with the `key` argument. You will see this pattern more in this course: they `key` argument can receive a _callable_ (here a _lambda_) which determines on what to sort, in this case number of stars.

> The lambda operator or lambda function is a way to create small anonymous functions, i.e. functions without a name. These functions are throw-away functions, i.e. they are just needed where they have been created. Lambda functions are mainly used in combination with the functions filter(), map() and reduce() [source](https://www.python-course.eu/lambda.php)

In [9]:
pb.get_repos()

In [6]:
Repo = namedtuple('Repo', 'name stars forks')

In [8]:
def get_repo_stats(user, n=5):
    """We did this exercise in our own 100 Days of Code:
       https://github.com/pybites/100DaysOfCode/blob/master/084/ghstats.py"""
    repos = []
    for repo in user.get_repos():
        if repo.fork:
            continue
        
        repos.append(Repo(name=repo.name,
                          stars=repo.stargazers_count,
                          forks=repo.forks_count))
    
    return sorted(repos, key=lambda x: x.stars, reverse=True)[:n]

In [12]:
get_repo_stats(pb)

[Repo(name='challenges', stars=393, forks=1327),
 Repo(name='100DaysOfCode', stars=306, forks=120),
 Repo(name='blog_code', stars=20, forks=14),
 Repo(name='bday-app', stars=19, forks=8),
 Repo(name='karmabot', stars=14, forks=12)]

We can call it on any user, let's see Mike's most popular repos:

In [13]:
mk = gh.get_user('mikeckennedy')
get_repo_stats(mk)

[Repo(name='write-pythonic-code-demos', stars=512, forks=138),
 Repo(name='python-jumpstart-course-demos', stars=510, forks=327),
 Repo(name='python-for-entrepreneurs-course-demos', stars=188, forks=101),
 Repo(name='python-switch', stars=140, forks=8),
 Repo(name='mongodb-quickstart-course', stars=92, forks=73)]

#### Example 2: auto-create a gist

Let's [create a gist](https://developer.github.com/v3/gists/#create-a-gist) sharing the `get_repo_stats` function with the world :)

To post on behalf of your user you need to instantiate `Github` with your login and password or create an auth token. I did the latter. 

1. Go to your Github Settings and create a token:
    ![go to settings](images/token1.png)
    
2. Define a name and choose the type of access you want to grant:
    ![define access](images/token2.png)
    
3. Copy the token and store it somewhere safe, as best practice I stored it as a env variable called `GH_GIST_CREATE_TOKEN`:
    ![copy token](images/token3.png)

Let's load the token from the environment:

In [30]:
token = os.environ['GH_GIST_CREATE_TOKEN']

KeyError: 'GH_GIST_CREATE_TOKEN'

And instantiance a new `Github` object using the token:

In [18]:
gh = Github(token)
gh

In [20]:
gh.rate_limiting

(5000, 5000)

It was not too obvious how to create a gist. This is the route I took:

1. Q: docs, on what object can I call `create_gist`? A: on an [AuthenticatedUser](http://pygithub.readthedocs.io/en/latest/github_objects/AuthenticatedUser.html#github.AuthenticatedUser.AuthenticatedUser.create_gist) object
2. How do I get an `AuthenticatedUser`, it turned out calling `get_user` on the `Github` object (`gh` variable) we instantiated with the token above:


In [24]:
me = gh.get_user()
me

AuthenticatedUser(login=None)

In [25]:
me.create_gist()

TypeError: create_gist() missing 2 required positional arguments: 'public' and 'files'

Although that is a useful error message, I'd like to know all the arguments:

In [26]:
help(me.create_gist)

Help on method create_gist in module github.AuthenticatedUser:

create_gist(public, files, description=NotSet) method of github.AuthenticatedUser.AuthenticatedUser instance
    :calls: `POST /gists <http://developer.github.com/v3/gists>`_
    :param public: bool
    :param files: dict of string to :class:`github.InputFileContent.InputFileContent`
    :param description: string
    :rtype: :class:`github.Gist.Gist`



Awesome, let's create our gist. First let's store our code in a variable:

In [27]:
code = '''
from collections import namedtuple

Repo = namedtuple('Repo', 'name stars forks')


def get_repo_stats(user, n=5):
    """Takes a Github user object and returns the top n most popular repos by star count,
       skips forks."""
    repos = []
    for repo in user.get_repos():
        if repo.fork:
            continue

        repos.append(Repo(name=repo.name,
                          stars=repo.stargazers_count,
                          forks=repo.forks_count))
        
    return sorted(repos, key=lambda x: x.stars, reverse=True)[:n]
'''

In [29]:
me.create_gist(True,
               {"repo_stats.py": InputFileContent(code)},
               "Get GH user's most popular repos")

Gist(id="6fa2a1a630fbfbcb98f696b278fc9913")

And the result:

![result](images/my_gist.png)

#### Pro tip: using pdb to inspect GH objects

Here is another useful skill when working with APIs that are not 100% documented or confusing: use the standard library debugger: `pdb`. For example, I wanted to print pybites' gists but was not sure what attributes I could include. `pdb` to the rescue: breaking inside the loop I can quickly do a `dir` on the `gist` object to see its attributes and methods

For more info on this topic see [How to Use Pdb to Debug Your Code](https://pybit.es/pdb-debugger.html).

In [ ]:
for gist in gh.get_user('pybites').get_gists():
    print(f'{gist.description} ({gist.created_at})')
    print(f'https://gist.github.com/{gist.id}')
    import pdb; pdb.set_trace()

script to get timezone distribution of our slack community (2018-04-22 04:53:31)
https://gist.github.com/61a88eb1eaca7fa689b506696e24287f
> <ipython-input-31-bf90a3856355>(1)<module>()
-> for gist in gh.get_user('pybites').get_gists():
(Pdb) 
(Pdb) gist
Gist(id="61a88eb1eaca7fa689b506696e24287f")


In [ ]:
for gist in gh.get_user("pybites").get_gists():
    print(f'{gist.description} ({gist.created_at} | {gist.fork_of})')
    print(f'https://gist.github.com/{gist.id}')
    print()

In [ ]:
gh.get_emojis()['snake']

![wrapping it up](https://assets-cdn.github.com/images/icons/emoji/unicode/1f40d.png?v7)

## Second + third day: practice!

Now you got the basics down, it's time to start using the Github API with Python yourself, here are some ideas. Again you can use PyGithub or any other wrapper module, as long as you use Python :)

Some ideas:
- Make a [Github Profiler](https://github.com/clamytoe/Github-Profiler) (this project was submitted for our [Code Challenge 16 - Query Your Favorite API](https://pybit.es/codechallenge16.html)).
- [Code Challenge 38 - Build Your Own Hacktoberfest Checker With Bottle](https://pybit.es/codechallenge38.html) - nice exercise to list user PRs for a given month.
- Try to implement a GH login button like we did on [codechalleng.es](http://codechalleng.es). For this you will need to register an app with your GH account. With the token you get upon login you can commit code on behalf of the user, but be aware: _with great power comes great responsibility_.
- Be creative, have fun!

Some extra links:
- Check out the full Github API documentation [here](https://developer.github.com/v3/)
- [Module of the Week - Requests-cache for Repeated API Calls](https://pybit.es/requests-cache.html): useful in dev mode when you need to make repeated calls to an API without exhausting its rate limits.

### Time to share what you've accomplished!

Be sure to share your last couple of days work on Twitter or Facebook. Use the hashtag **#100DaysOfCode**.

Here are [some examples](https://twitter.com/search?q=%23100DaysOfCode) to inspire you. Consider including [@talkpython](https://twitter.com/talkpython) and [@pybites](https://twitter.com/pybites) in your tweets.

*See a mistake in these instructions? Please [submit a new issue](https://github.com/talkpython/100daysofcode-with-python-course/issues) or fix it and [submit a PR](https://github.com/talkpython/100daysofcode-with-python-course/pulls).*